In [49]:
import transformers
import torch
import pandas as pd
import json
import os

In [5]:
from huggingface_hub import login
login('login_key')

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)


Device: cuda


In [6]:
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="cuda", 
)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [27]:
messages = [
        {"role": "system", "content": "You are a summarizer. Extract summaries for defect, consequence, corrective actions, and Merged Summary combining all three previous, and discard irrelevant information and generate output like defect_summary: content ,consequence_summary: content,corrective_summary: content and merged_summary: content"}, 
        {"role": "user", "content": "22C12 VISIBILITY:WINDSHIELD Ford Motor Company   V 63294 20220718 MFR Ford Motor Company 20220624 20220624  212 Ford Motor Company (Ford) is recalling certain 2022 Bronco and Ranger vehicles.  The windshield may not have been properly bonded to the vehicle, which could allow it to detach during a crash.  As such, these vehicles fail to comply with the requirements of Federal Motor Vehicle Safety Standard number 212, \"Windshield Mounting.\" A windshield that separates from a vehicle during a crash can increase the risk of injury. Dealers will remove and reinstall the windshield, free of charge.  Owner notification letters were mailed July 18, 2022.  Owners may contact Ford customer service at 1-866-436-7332.  Ford's number for this recall is 22C12.   Owners may also contact the National Highway Traffic Safety Administration Vehicle Safety Hotline at 1-888-327-4236 (TTY 1-800-424-9153), or go to www.nhtsa.gov.  000121748005192749000000236 Windshield Bronco Windshield N2DB-S03100-*"}, 
    ]

    # Generate the summary using the pipeline
outputs = pipeline(
        messages,
        max_new_tokens=256,
    )
    
    # Extract individual summaries from the generated text
generated_text = outputs[0]["generated_text"]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [28]:
print(generated_text[-1])

{'role': 'assistant', 'content': 'Based on the given information, here are the extracted summaries:\n\n**defect_summary:** The windshield may not have been properly bonded to the vehicle, which could allow it to detach during a crash.\n\n**consequence_summary:** A windshield that separates from a vehicle during a crash can increase the risk of injury.\n\n**corrective_summary:** Dealers will remove and reinstall the windshield, free of charge.\n\n**merged_summary:** Ford is recalling certain 2022 Bronco and Ranger vehicles due to a defect in the windshield bonding, which can increase the risk of injury during a crash. Dealers will remove and reinstall the windshield free of charge to correct the issue.'}


In [93]:
def summarize_entry(entry):
    messages = [
        {"role": "system", "content": "You are a summarizer. Extract summaries for defect, consequence, corrective actions, and Merged Summary combining all three previous in the following format **defect_summary:** content ,**consequence_summary:** content,**corrective_summary:** content and **merged_summary:** content  and discard irrelevant information  "}, 
        {"role": "user", "content": entry}, 
    ]
    # Generate the summary using the pipeline
    outputs = pipeline(
        messages,
        max_new_tokens=256,
    )
    
    # Extract individual summaries from the generated text
    generated_text = outputs[0]["generated_text"][-1]['content']
    summaries = {
        "defect_summary": "", 
        "consequence_summary": "", 
        "corrective_summary": "", 
        "merged_summary": ""
    }

    # Populate summaries based on generated text
    for line in generated_text.splitlines():
        if line.startswith("**defect_summary:** "):
            summaries["defect_summary"] = line.replace("**defect_summary:** ", "").strip()
        elif line.startswith("**consequence_summary:** "):
            summaries["consequence_summary"] = line.replace("**consequence_summary:** ", "").strip()
        elif line.startswith("**corrective_summary:** "):
            summaries["corrective_summary"] = line.replace("**corrective_summary:** ", "").strip()
        elif line.startswith("**merged_summary:** "):
            summaries["merged_summary"] = line.replace("**merged_summary:** ", "").strip()

    
    return summaries


In [94]:
input_file = '/kaggle/input/predii-data-1/dataset.json'
with open(input_file, 'r') as f:
    json_data = json.load(f)

In [95]:
output_file = '/kaggle/working/database_chatbox.json'
checkpoint_interval = 10  # Save every 10 entries
processed_data = []

In [96]:
if os.path.exists(output_file):
    with open(output_file, 'r') as f:
        processed_data = json.load(f)
    start_index = len(processed_data)
else:
    start_index = 0

In [ ]:
for i, entry in enumerate(json_data[start_index:], start=start_index):
    summaries = summarize_entry(entry['Other info'])
    processed_entry = {
        "make": entry["make"],
        "model": entry["model"],
        "year": entry["year"],
        "defect_summary": summaries["defect_summary"],
        "consequence_summary": summaries["consequence_summary"],
        "corrective_summary": summaries["corrective_summary"],
        "merged_summary": summaries["merged_summary"]
    }
    processed_data.append(processed_entry)

    # Checkpoint save every `checkpoint_interval` entries
    if (i + 1) % checkpoint_interval == 0:
        with open(output_file, 'w') as f:
            json.dump(processed_data, f, indent=2)
        print(f"Checkpoint saved at entry {i + 1}")

# Final save after completing all entries
with open(output_file, 'w') as f:
    json.dump(processed_data, f, indent=2)

print("Final summarized data saved to", output_file)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Checkpoint saved at entry 10


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Checkpoint saved at entry 20


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

Checkpoint saved at entry 3420


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Checkpoint saved at entry 3430


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Checkpoint saved at entry 3440
